# Building a Chocolate Quality Classifier using Transfer Learning

In this tutorial, we'll walk through building a deep learning system that can classify chocolate quality using transfer learning with ResNet50. We'll cover:

1. Data preparation and loading
2. Understanding transfer learning with ResNet
3. Training process and visualization
4. Model evaluation and interpretation

Let's start by importing our dependencies!

In [ ]:
import torch
from torchvision import transforms, models
import matplotlib.pyplot as plt
from dataset import PoopDataset, PoopClassifier
import torch.nn as nn
import numpy as np
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Understanding the Dataset

Our dataset consists of chocolate images organized into 7 quality classes. Let's examine our data structure and visualize some examples.

In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load dataset
dataset = PoopDataset("data", transform=transform)
print(f"Total number of images: {len(dataset)}")
print(f"Classes: {dataset.classes}")

Let's visualize some example images from our dataset:

In [ ]:
def show_images(dataset, num_images=5):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    for i in range(num_images):
        img, label = dataset[i]
        # Denormalize
        img = img.permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        
        axes[i].imshow(img)
        axes[i].set_title(f'Class {dataset.classes[label]}')
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

show_images(dataset)

## 2. Transfer Learning with ResNet50

We're using a pre-trained ResNet50 model and adapting it for our chocolate classification task. Let's understand how this works:

In [ ]:
def visualize_model_architecture():
    model = PoopClassifier(num_classes=7)
    print("Model Architecture:")
    print("\nBase Model (ResNet50):")
    print(" - Pre-trained on ImageNet (1000 classes)")
    print(" - Frozen layers (not updated during training)")
    print("\nOur Modifications:")
    print(f" - Final layer adapted for {len(dataset.classes)} classes")
    print(" - Only training the final layer")
    
    return model

model = visualize_model_architecture()

## 3. Understanding the Training Process

Let's break down how the model learns:

In [ ]:
def explain_training_process():
    print("Training Process Steps:")
    print("1. Forward Pass:")
    print("   - Image → ResNet50 → Feature Vector → Classifier → Prediction")
    print("\n2. Loss Calculation:")
    print("   - Compare prediction with true label")
    print("   - Calculate how wrong the model was")
    print("\n3. Backward Pass:")
    print("   - Calculate gradients")
    print("   - Update model weights")
    print("\n4. Validation:")
    print("   - Check performance on unseen data")
    print("   - Save best performing model")

explain_training_process()

## 4. Visualizing Feature Maps

Let's see how the model "sees" chocolate images at different layers:

In [ ]:
def get_feature_maps(model, image):
    # Get feature maps from different layers
    features = {}
    def hook_fn(module, input, output, name):
        features[name] = output
    
    # Register hooks
    hooks = [
        model.base_model.conv1.register_forward_hook(
            lambda m, i, o: hook_fn(m, i, o, 'conv1')
        ),
        model.base_model.layer1[0].conv1.register_forward_hook(
            lambda m, i, o: hook_fn(m, i, o, 'layer1')
        )
    ]
    
    # Forward pass
    with torch.no_grad():
        model(image.unsqueeze(0))
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    return features

# Get a sample image
image, _ = dataset[0]
features = get_feature_maps(model, image)

# Visualize features
plt.figure(figsize=(15, 5))
for i, (name, feat) in enumerate(features.items()):
    plt.subplot(1, len(features), i+1)
    plt.imshow(feat[0, 0].cpu().numpy())
    plt.title(f'{name} features')
    plt.axis('off')
plt.tight_layout()
plt.show()

## 5. Making Predictions

Finally, let's see how the model makes predictions on new images:

In [ ]:
def predict_image(model, image_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor.unsqueeze(0))
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# Get a test image
test_image, true_label = dataset[10]
predicted_class = predict_image(model, test_image)

# Show results
plt.imshow(test_image.permute(1, 2, 0).numpy())
plt.title(f'Predicted: Class {predicted_class}, True: Class {true_label}')
plt.axis('off')
plt.show()

## Conclusion

We've built and explained a complete chocolate classification system using transfer learning. Key takeaways:

1. Transfer learning lets us leverage pre-trained models
2. ResNet50's deep architecture captures complex features
3. We only needed to train the final classification layer
4. The model learns hierarchical features, from simple edges to complex patterns

This approach is particularly effective for specialized classification tasks with limited data.